In [71]:
import pandas as pd
import numpy as np


from data_loader import load_from_tsfile_to_dataframe
from regressor_tools import process_data
import mlflow
from tsfeatures import tsfeatures
#np.set_printoptions(threshold=np.inf)
#pd.set_option('display.max_rows', None)  
#pd.set_option('display.max_columns', None) 


Conclusions:
- In Datasets with missing values not all datapoints are loaded to test and train set. Around 0.1% to 7% can be missing. 
The datapoints that are loaded are fine and complete and contain the missnig values. (One Datapoint is one target value and the belonging ts's for each dimesnion. E.g a single (x,y) tupel.

- Seems to be the load_from_ts_file function. Very long and large. process_data works fine and has no o data loss
- Possible further steps: 
-> Try loading with sktime or other similar thing from directly from .ts file. 
-> Check out .ts file and see if there the data is already missing.
-> Check out big loading function.

- But: For no not important. Only few datasets have missing Datapoints. Everything should work well without that. Takes some time finding the problem.


Facts:
- Sets with no missing values:
- IEEEPG, ApliancesEnergy, FloodModelling have Full Data Loading. ( Maybe 1 or 2 Datasets are missing sometimes sporadically.)

- Sets with missing values:
- BejingPM25Quality  12432/11918 (4% is missing)  5445/5048, (7% is missing)
- Household loads 746/745 und 694/686 -> 0.1%
- Benzene  3433/3349	5445/5163  -> around 5%
-> Between 1% to 7% is missing when loading datasets with missing values.

- In Bejing 2500, Benzene, Houshold datasets we have missing values. Means in a single ts some values of a timestep are nan.


Understanding:

- load_from_tsfile 
- returns a nested df. columnindex are the dimensions. The rowindex are the datapoints. 
One entry is a pd series of the ts belonging to the dim and datapoint. The index of the series is the datetime and the value is the value of the ts at this datetime.

- process_data:
- After processing we get a np cubicle: Each layer is one datapoint. In each layer one column form top to down is a ts to the belonging dimension.
- Interpolates the nan values. Applies normalisiation either none, standart or minmax. (and scales for each ts the dimension to same length, if different length, then chops eacht ts to length of min_length)

- min_length: checks minimum length of all the ts in the df. process_data cuts then all ts to the min_len! In our Datasets all ts have the same length! Keep Function just for safety, they say in paper some dimensions have uneqal length!


Unclear:
- Why not all datapoints are loaded with load_from_tsfile
-(Why Benzene larger Testset than Traininset)


In [72]:

#data_train_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/HouseholdPowerConsumption1_TRAIN.ts"
#data_test_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/HouseholdPowerConsumption1_TEST.ts"

#data_train_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/FloodModeling1_TRAIN.ts"
#data_test_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/FloodModeling1_TEST.ts"

data_train_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/BenzeneConcentration_TRAIN.ts"
data_test_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/BenzeneConcentration_TEST.ts"

#data_train_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/BeijingPM25Quality_TRAIN.ts"
#data_test_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/BeijingPM25Quality_TEST.ts"

#data_train_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/AppliancesEnergy_TRAIN.ts"
#data_test_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/AppliancesEnergy_TEST.ts"

#data_train_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/IEEEPPG_TRAIN.ts"
#data_test_path = "/home/sim/Desktop/TS Extrinsic Regression/Checking Out Data/data/IEEEPPG_TEST.ts"


train_x, train_y = load_from_tsfile_to_dataframe(data_train_path, replace_missing_vals_with='NaN')
test_x, test_y = load_from_tsfile_to_dataframe(data_test_path, replace_missing_vals_with='NaN')




3375it [00:48, 69.39it/s]
5189it [01:13, 70.55it/s]


In [73]:
print(train_x.shape)
print(test_x.shape)
print(train_x.columns)
print(train_x.info())
print(train_x["dim_0"].info)
print(train_x["dim_0"][0].info)

(3349, 8)
(5163, 8)
Index(['dim_0', 'dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', 'dim_6', 'dim_7'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dim_0   3349 non-null   object
 1   dim_1   3349 non-null   object
 2   dim_2   3349 non-null   object
 3   dim_3   3349 non-null   object
 4   dim_4   3349 non-null   object
 5   dim_5   3349 non-null   object
 6   dim_6   3349 non-null   object
 7   dim_7   3349 non-null   object
dtypes: object(8)
memory usage: 209.4+ KB
None
<bound method Series.info of 0       2004-03-10 18:00:00    1360.000000
2004-03-10 ...
1       2004-03-10 19:00:00    1292.250000
2004-03-10 ...
2       2004-03-10 20:00:00    1402.000000
2004-03-10 ...
3       2004-03-10 21:00:00    1375.500000
2004-03-10 ...
4       2004-03-10 22:00:00    1272.250000
2004-03-10 ...
5       2004-03-10 23:00:00    1197.000000
2004-03-1

In [74]:
# Count NaN values in Df
nan_counts = 0


for index, row in train_x.iterrows():
    for column_name, value in row.items():
        nan_counts = nan_counts + value.isna().sum()

print(nan_counts)

144472


In [75]:
# Processing Data
min_len = np.inf
for i in range(len(train_x)):
    x = train_x.iloc[i, :]
    all_len = [len(y) for y in x]
    min_len = min(min(all_len), min_len)

for i in range(len(test_x)):
    x = test_x.iloc[i, :]
    all_len = [len(y) for y in x]
    min_len = min(min(all_len), min_len)


print(min_len)

# Min_Len checks shortest length of time series inside the DF. In our Datasets i think every ts has same length!

240


In [76]:
# Processing Data
min_len = np.inf
for i in range(len(train_x)):
    x = train_x.iloc[i, :]
    all_len = [len(y) for y in x]
    min_len = min(min(all_len), min_len)

for i in range(len(test_x)):
    x = test_x.iloc[i, :]
    all_len = [len(y) for y in x]
    min_len = min(min(all_len), min_len)

    

train_x_p= process_data(train_x, normalise=None, min_len=min_len)
test_x_p = process_data(test_x, normalise=None, min_len=min_len)

print(train_x_p.shape)



100%|██████████| 5163/5163 [00:05<00:00, 1008.89it/s]

(3349, 240, 8)


In [77]:
# Proof for Appliances that no missing values after processing

print(train_x['dim_0'][3])
print(train_x_p[3,:,0])

2004-03-10 21:00:00    1375.500000
2004-03-10 22:00:00    1272.250000
2004-03-10 23:00:00    1197.000000
2004-03-11 00:00:00    1185.000000
2004-03-11 01:00:00    1136.250000
2004-03-11 02:00:00    1094.000000
2004-03-11 03:00:00    1009.750000
2004-03-11 04:00:00    1011.000000
2004-03-11 05:00:00    1066.000000
2004-03-11 06:00:00    1051.750000
2004-03-11 07:00:00    1144.000000
2004-03-11 08:00:00    1333.250000
2004-03-11 09:00:00    1351.000000
2004-03-11 10:00:00    1233.250000
2004-03-11 11:00:00    1178.750000
2004-03-11 12:00:00    1236.000000
2004-03-11 13:00:00    1285.500000
2004-03-11 14:00:00    1371.000000
2004-03-11 15:00:00    1310.000000
2004-03-11 16:00:00    1291.750000
2004-03-11 17:00:00    1383.000000
2004-03-11 18:00:00    1580.750000
2004-03-11 19:00:00    1775.500000
2004-03-11 20:00:00    1640.000000
2004-03-11 21:00:00    1312.750000
2004-03-11 22:00:00     964.500000
2004-03-11 23:00:00     912.750000
2004-03-12 00:00:00    1080.250000
2004-03-12 01:00:00 